In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

from openpyxl import Workbook

In [16]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
url = "https://www.naver.com/"
driver.get(url)

In [17]:
def returnText(selector):
    try:
        text = driver.find_element(By.CSS_SELECTOR, selector).text
    except:
        text = None
    return(text)

In [18]:
searchBox = driver.find_element(By.XPATH,'//*[@id="query"]')
searchBox.send_keys("식당")
time.sleep(1)
searchBox.send_keys(Keys.RETURN)
map = driver.find_element(By.XPATH,'//*[@id="lnb"]/div[1]/div/div[1]/div/div[1]/div[1]/a').click()
time.sleep(1)

In [19]:
driver.switch_to.window(driver.window_handles[-1])
time.sleep(1)
iFrame = driver.find_element(By.ID,'searchIframe')
driver.switch_to.frame(iFrame)
time.sleep(0.3)


In [20]:
restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')

last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
while True:
    driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
    new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
    if new_height == last_height:
        break
    last_height = new_height
    time.sleep(0.1)

In [21]:
restaurants = []
i = 1
time.sleep(1)
while True:
    driver.switch_to.default_content()
    iFrame = driver.find_element(By.ID,'searchIframe')
    driver.switch_to.frame(iFrame)
    currentPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
    try: 
        restaurant = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[%d]/div[1]/a/div/div/span[1]'%i)
        restaurant.click()
        time.sleep(1)
        driver.switch_to.default_content()
        iFrame = driver.find_element(By.ID,'entryIframe')
        driver.switch_to.frame(iFrame)
        restaurantName = returnText('#_title > div > span.GHAhO')
        restaurantTags = returnText('#_title > div > span.lnJFt')
        restaurantRating = returnText('#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span.PXMot.LXIwF')
        time.sleep(1)
        restaurantAddress = returnText('#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH')
        restaurantCallNumber = returnText('#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.nbXkr > div > span.xlx7Q')
        restaurants.append({
            'name': restaurantName,
            'tags': restaurantTags,
            'rating': restaurantRating,
            'address': restaurantAddress,
            'callNumber': restaurantCallNumber
        })
        i+=1
    except:
        nextButton = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[6]')
        nextButton.click()
        time.sleep(1)
        nextPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
        if nextPage == currentPage: break
        restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')
        last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
        while True:
            driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
            new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
            if new_height == last_height:
                break
            last_height = new_height
            time.sleep(0.2)
        i = 1
    print(restaurants[len(restaurants)-1])

{'name': '매취랑 용인외대점', 'tags': '감자탕', 'rating': None, 'address': '경기 용인시 처인구 모현읍 외대로26번길 2 1층 102호', 'callNumber': '031-323-6618'}
{'name': '제주도마고기 모현점', 'tags': '육류,고기요리', 'rating': None, 'address': '경기 용인시 처인구 모현읍 백옥대로2332번길 21-13 104호, 105호', 'callNumber': '0507-1354-5892'}
{'name': '삼부자생고기', 'tags': '육류,고기요리', 'rating': None, 'address': '경기 용인시 처인구 모현읍 백옥대로 2324', 'callNumber': '0507-1483-3389'}
{'name': '왕산돈까스', 'tags': '돈가스', 'rating': '별점\n4.45', 'address': '경기 용인시 처인구 모현읍 백옥대로 2349 1층', 'callNumber': '031-333-3350'}
{'name': '똠팔이 용인점', 'tags': '바닷가재요리', 'rating': None, 'address': '경기 용인시 처인구 모현읍 갈월로 8 지하1층', 'callNumber': '0507-1441-0836'}
{'name': '황태해장국', 'tags': '해장국', 'rating': '별점\n4.28', 'address': '경기 용인시 처인구 모현읍 백옥대로 2229', 'callNumber': '0507-1442-5535'}
{'name': '엄마손칼국수', 'tags': '칼국수,만두', 'rating': '별점\n4.43', 'address': '경기 용인시 처인구 모현읍 백옥대로2384번길 18', 'callNumber': '031-333-8917'}
{'name': '미가초밥', 'tags': '초밥,롤', 'rating': '별점\n4.39', 'address': '경기 용인시 처인구 모현읍 백옥대로2

In [23]:
wb = Workbook()
ws = wb.active
ws.title = "Restaurants"
column = ['name','tags','rating','adress','callNumber']
ws.append(column)
for i in range(len(restaurants)):
    row = [restaurants[i]['name'],restaurants[i]['tags'],restaurants[i]['rating'],restaurants[i]['address'],restaurants[i]['callNumber']]
    ws.append(row)
wb.save('restaurant list.xlsx')
wb.close()